<a href="https://colab.research.google.com/github/brs1977/frame-parsing/blob/master/frame_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
#mystem3
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-03-02 05:10:00--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.245, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-03-02 05:10:01--  http://cache-mskm908.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)... 5.45.220.18, 2a02:6b8:0:2002::19
Connecting to cache-mskm908.cdn.yandex.net (cache-mskm908.cdn.yandex.net)|5.45.220.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [2]:
#udpipe
!git clone http://github.com/ufal/udpipe
%cd udpipe/src
!make 
!cp udpipe /bin
!udpipe --help

Cloning into 'udpipe'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 4030 (delta 56), reused 91 (delta 37), pack-reused 3894
Receiving objects: 100% (4030/4030), 1.81 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (2824/2824), done.
/content/udpipe/src
g++ -o .build/udpipe.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c udpipe.cpp
g++ -o .build/utils-options.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c utils/options.cpp
g++ -o .build/morphodita-derivator-derivation_formatter.linux-gcc-normal.o -MMD -MP -std=c++11 -W -Wall -mtune=generic -msse -msse2 -mfpmath=sse -fvisibility=hidden -U_FORTIFY_SOURCE -O3 -I. -c morphodita/derivator/derivation_formatter.cpp
g++ -o .build/morphodita-derivator-derivator_

/content/udpipe/src


In [8]:
#!cp /content/drive/My\ Drive/nlp/udpipe /bin

%cd /content
!pip install conllu
!git clone https://github.com/brs1977/frame-parsing

%cd /content/frame-parsing


/content
Cloning into 'frame-parsing'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 109 (delta 3), reused 11 (delta 3), pack-reused 98
Receiving objects: 100% (109/109), 21.58 MiB | 27.63 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/frame-parsing


In [0]:
# !cp /content/udpipe/src/udpipe /content/drive/My\ Drive/nlp
# %cd /content/frame-parsing

/content/frame-parsing


In [0]:

!git pull origin master



#Parsing

In [9]:
# del features
from create_features import features
from sklearn.externals import joblib
from pymystem3 import Mystem
from collections import OrderedDict
from string import whitespace, punctuation

clf = joblib.load('frame_parser.pkl')
vec = joblib.load('feature_transformer.pkl')

def frames(sentence):
    header = ('word', 'lex', 'pos', 'gram', 'prev_gr', 'prev_lex', 'rel', 'pred_lemma')
    anas = m.analyze(sentence)
    fr = OrderedDict()
    data = OrderedDict()
    for w in anas:
        word = w['text']
        #if word in whitespace or word in punctuation:
        #    continue
        try:
            gr = w['analysis'][0]['gr']
            lex = w['analysis'][0]['lex']
        except:
            gr, lex = None, None
        data[word] = [lex, gr, None, None, None, None]
    vector = features(data)
    vectors = OrderedDict((k, v) for k,v in vector.items() if k not in whitespace and k not in punctuation)
    for w in vectors:
        feats = [w] + [str(x) for x in vectors[w]][:-1]
        feats = dict(zip(header, feats))
        v = vec.transform(feats)
        role = clf.predict(v)[0]
        fr[w] = role
    #print(fr)
    return fr
    #vector = vec.transform(features)
    #return clf.predict(vector)[0][0]



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.stochastic_gradient module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.sgd_fast modul

In [20]:
m = Mystem('/bin/mystem')
#sentence = 'мама мыла раму'
sentence = 'Публичное акционерное общество Мвидео именуемое в дальнейшем Общество является коммерческой организацией созданной в соответствии с Гражданским кодексом Российской Федерации Федеральным законом «Об акционерных обществах» № 208-ФЗ (далее – Закон) и иными нормативными правовыми актами Российской Федерации в целях извлечения прибыли и обладает полной хозяйственной самостоятельностью общество является юридическим лицом и действует на основании настоящего устава и законодательства российской федерации'
frames(sentence)


OrderedDict([('id', 16), ('form', 'кодексом'), ('lemma', 'кодекЕС'), ('upostag', 'NOUN'), ('xpostag', 'NN'), ('feats', OrderedDict([('Animacy', 'Anim'), ('Case', 'Ins'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 3), ('form', 'общество'), ('lemma', 'ОБЩЕСТВО'), ('upostag', 'NOUN'), ('xpostag', 'NN'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 16), ('deprel', 'nsubj'), ('deps', None), ('misc', None)])
OrderedDict([('id', 1), ('form', 'Публичное'), ('lemma', 'Публичное'), ('upostag', 'PROPN'), ('xpostag', 'NNP'), ('feats', OrderedDict([('Animacy', 'Anim'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 3), ('deprel', 'nsubj'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s\\s')]))])
OrderedDict([('id', 2), ('form', 'акционерное'), ('lemma', 'акционЕРНЫЙ'), ('upostag', 'ADJ'), ('xpostag', 'JJL'), ('

OrderedDict([('Публичное', 'пациенс'),
             ('акционерное', 'пациенс'),
             ('общество', 'конечная точка'),
             ('Мвидео', 'экспериенцер'),
             ('именуемое', 'конечная точка'),
             ('в', 'конечная точка'),
             ('дальнейшем', 'пациенс'),
             ('Общество', 'экспериенцер'),
             ('является', 'экспериенцер'),
             ('коммерческой', 'конечная точка'),
             ('организацией', 'конечная точка'),
             ('созданной', 'конечная точка'),
             ('соответствии', 'конечная точка'),
             ('с', 'конечная точка'),
             ('Гражданским', 'ресурс'),
             ('кодексом', 'содержание высказывания'),
             ('Российской', 'конечная точка'),
             ('Федерации', 'среда'),
             ('Федеральным', 'конечная точка'),
             ('законом', 'адресат'),
             (' «', 'содержание действия'),
             ('Об', 'пациенс'),
             ('акционерных', 'адресат'),
             

In [21]:
sentence = 'Публичное акционерное общество «М.видео» создано в результате реорганизации в форме преобразования Общества с ограниченной ответственностью «Компания «М.видео» на основании решения Общего собрания участников ООО «Компания «М.видео» (Протокол №14/2006 от «01» июля 2006 года).'
frames(sentence)

OrderedDict([('id', 9), ('form', 'создано'), ('lemma', 'СОЗДАТЬ'), ('upostag', 'VERB'), ('xpostag', 'VBNH'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Aspect', 'Perf'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing'), ('Tense', 'Past'), ('Variant', 'Short'), ('VerbForm', 'Part'), ('Voice', 'Pass')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 3), ('form', 'общество'), ('lemma', 'ОБЩЕСТВО'), ('upostag', 'NOUN'), ('xpostag', 'NN'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 9), ('deprel', 'nsubj:pass'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s')]))])
OrderedDict([('id', 1), ('form', 'Публичное'), ('lemma', 'Публичное'), ('upostag', 'PROPN'), ('xpostag', 'NNP'), ('feats', OrderedDict([('Animacy', 'Anim'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 3), ('deprel', 'nsubj'), ('deps', None), ('misc', OrderedDict([('SpacesA

OrderedDict([('Публичное', 'пациенс'),
             ('акционерное', 'пациенс'),
             ('общество', 'конечная точка'),
             (' «', 'экспериенцер'),
             ('М', 'пациенс'),
             ('видео', 'пациенс'),
             ('» ', 'конечная точка'),
             ('создано', 'пациенс'),
             ('в', 'конечная точка'),
             ('результате', 'пациенс'),
             ('реорганизации', 'конечная точка'),
             ('форме', 'конечная точка'),
             ('преобразования', 'конечная точка'),
             ('Общества', 'конечная точка'),
             ('с', 'экспериенцер'),
             ('ограниченной', 'ресурс'),
             ('ответственностью', 'конечная точка'),
             ('Компания', 'причина'),
             ('на', 'экспериенцер'),
             ('основании', 'содержание действия'),
             ('решения', 'конечная точка'),
             ('Общего', 'признак'),
             ('собрания', 'экспериенцер'),
             ('участников', 'экспериенцер'),
      

In [22]:
sentence = '1.2. Фирменное наименование Общества: на русском языке: полное: Публичное акционерное общество «М.видео»; сокращенное: ПАО «М.видео»; на английском языке: полное: Public Joint-Stock Company "M.video"; сокращенное: PJSC "M.video".'
frames(sentence)

OrderedDict([('id', 4), ('form', 'наименование'), ('lemma', 'НАИМЕНОВАНИЕ'), ('upostag', 'NOUN'), ('xpostag', 'NN'), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Acc'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', None)])
OrderedDict([('id', 1), ('form', '1.2'), ('lemma', '1.2'), ('upostag', 'NUM'), ('xpostag', 'CD'), ('feats', OrderedDict([('Case', 'Nom'), ('NumType', 'Card')])), ('head', 4), ('deprel', 'nummod'), ('deps', None), ('misc', None)])
OrderedDict([('id', 3), ('form', 'Фирменное'), ('lemma', 'Фирменное'), ('upostag', 'PROPN'), ('xpostag', 'NNP'), ('feats', OrderedDict([('Animacy', 'Anim'), ('Case', 'Nom'), ('Gender', 'Masc'), ('Number', 'Sing')])), ('head', 4), ('deprel', 'nmod'), ('deps', None), ('misc', OrderedDict([('SpacesAfter', '\\s\\s\\s')]))])
OrderedDict([('id', 2), ('form', '.'), ('lemma', '.'), ('upostag', 'PUNCT'), ('xpostag', '.'), ('feats', None), ('head', 3), ('deprel', 'punct'), ('deps', None),

OrderedDict([('1.2', 'пациенс'),
             ('. ', 'пациенс'),
             ('Фирменное', 'пациенс'),
             ('наименование', 'пациенс'),
             ('Общества', 'конечная точка'),
             (': ', 'экспериенцер'),
             ('на', 'пациенс'),
             ('русском', 'содержание действия'),
             ('языке', 'потенциальный пациенс'),
             ('полное', 'экспериенцер'),
             ('Публичное', 'результат'),
             ('акционерное', 'конечная точка'),
             ('общество', 'конечная точка'),
             (' «', 'экспериенцер'),
             ('М', 'пациенс'),
             ('видео', 'пациенс'),
             ('»; ', 'конечная точка'),
             ('сокращенное', 'пациенс'),
             ('ПАО', 'конечная точка'),
             ('английском', 'конечная точка'),
             ('Public', 'агенс'),
             ('Joint', 'пациенс'),
             ('Stock', 'пациенс'),
             ('Company', 'пациенс'),
             (' "', 'пациенс'),
             ('M', 'пац

In [0]:
Нужна другая токенизация (не выделяет даты, номера пунктов, неправильно обрабатывает спец символы, не обрабатывает несколько предложений)